In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split
from sagemaker.tensorflow import TensorFlow

# Train Model

In [28]:
ds_data = np.load('dataset3/data.npy', allow_pickle=True)
ds_labels = np.load('dataset3/labels.npy', allow_pickle=True)
x_train, x_val, y_train, y_val = train_test_split(ds_data, ds_labels, train_size=0.75)

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

prefix = 'ml-screen-reader'
training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

tf_estimator = TensorFlow(entry_point='model_builders/dnn.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          framework_version='2.3', 
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 20,
                              'batch-size': 256,
                              'learning-rate': 0.01}
                         )
# tf_estimator = TensorFlow(entry_point='model_builders/dnn.py', 
#                           role=role,
#                           instance_count=1, 
#                           instance_type='local',
#                           framework_version='1.12', 
#                           py_version='py3',
#                           script_mode=True,
#                           hyperparameters={'epochs': 1}
#                          )

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

s3://sagemaker-us-east-2-435269695145/ml-screen-reader/training/training.npz
s3://sagemaker-us-east-2-435269695145/ml-screen-reader/validation/validation.npz
2021-03-25 08:11:11 Starting - Starting the training job...
2021-03-25 08:11:34 Starting - Launching requested ML instancesProfilerReport-1616659870: InProgress
......
2021-03-25 08:12:36 Starting - Preparing the instances for training...
2021-03-25 08:13:10 Downloading - Downloading input data...
2021-03-25 08:13:35 Training - Downloading the training image..2021-03-25 08:13:45.132458: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03-25 08:13:45.136075: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-03-25 08:13:45.283373: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03

# Load Model

In [ ]:
MODEL_DIRECTORY = 'models'
MODEL = ''
VERSION = ''
tf_estimator = tf.saved_model.load(os.path.join(MODEL_DIRECTORY, MODEL, VERSION))

# Deploy Model

In [29]:
tf_endpoint_name = 'ml-screen-reader-endpoint'
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                  instance_type='ml.m4.xlarge',
                                  endpoint_name=tf_endpoint_name)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-us-east-2-435269695145/tensorflow-training-2021-03-25-08-11-10-751/output/model.tar.gz.

In [ ]:
tf_endpoint_name = 'ml-screen-reader-endpoint'
sess.delete_endpoint(endpoint_name=tf_endpoint_name)